# Value Iteration and the Bellman Equation

This notebook demonstrates the core Bellman equation solver on a simple GridWorld MDP.
We compare **Value Iteration**, **Policy Iteration**, and **Modified Policy Iteration**,
visualizing their convergence behavior.

In [ ]:
import numpy as np
import bellmaneq
from bellmaneq.viz import plot_convergence, plot_solver_comparison
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

## Constructing a GridWorld MDP

We define a 4x4 GridWorld with:
- 16 states (positions on the grid)
- 4 actions (up, down, left, right)
- State 15 (bottom-right) is the goal with reward +1
- All other transitions yield reward -0.04 (step penalty)
- Movements that would leave the grid keep the agent in place

In [ ]:
GRID_SIZE = 4
N_STATES = GRID_SIZE * GRID_SIZE
N_ACTIONS = 4  # up, down, left, right
GOAL = N_STATES - 1
STEP_PENALTY = -0.04
GOAL_REWARD = 1.0

# Action deltas: up=(-1,0), down=(+1,0), left=(0,-1), right=(0,+1)
DELTAS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

def to_state(row, col):
    return row * GRID_SIZE + col

def from_state(s):
    return divmod(s, GRID_SIZE)

# Build reward and transition matrices
rewards = np.full((N_STATES, N_ACTIONS), STEP_PENALTY)
transitions = np.zeros(N_STATES * N_ACTIONS * N_STATES)

for s in range(N_STATES):
    row, col = from_state(s)
    for a, (dr, dc) in enumerate(DELTAS):
        nr, nc = row + dr, col + dc
        if 0 <= nr < GRID_SIZE and 0 <= nc < GRID_SIZE:
            ns = to_state(nr, nc)
        else:
            ns = s  # stay in place
        transitions[s * N_ACTIONS * N_STATES + a * N_STATES + ns] = 1.0
        if ns == GOAL:
            rewards[s, a] = GOAL_REWARD

print(f'MDP: {N_STATES} states, {N_ACTIONS} actions')
print(f'Reward matrix shape: {rewards.shape}')
print(f'Transition tensor elements: {len(transitions)}')

## Solving with Value Iteration

The Bellman optimality equation:

$$V^*(s) = \max_a \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a) V^*(s') \right]$$

Value Iteration repeatedly applies the Bellman operator $T$:
$$V_{k+1} = TV_k$$

By the Banach fixed-point theorem, this converges geometrically at rate $\gamma$.

In [ ]:
gamma = 0.95

vi_result = bellmaneq.solve_value_iteration(rewards, transitions, gamma=gamma, tol=1e-10)

print(f'Converged: {vi_result.converged}')
print(f'Iterations: {vi_result.iterations}')

# Reshape values into the grid
values = vi_result.get_values().reshape(GRID_SIZE, GRID_SIZE)
policy = vi_result.get_policy().reshape(GRID_SIZE, GRID_SIZE)

print('\nOptimal Value Function:')
print(np.round(values, 2))

ACTION_SYMBOLS = ['\u2191', '\u2193', '\u2190', '\u2192']  # up, down, left, right
print('\nOptimal Policy:')
for r in range(GRID_SIZE):
    row_str = ''
    for c in range(GRID_SIZE):
        if to_state(r, c) == GOAL:
            row_str += ' G '
        else:
            row_str += f' {ACTION_SYMBOLS[policy[r, c]]} '
    print(row_str)

## Visualizing the Value Function

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Value function heatmap
im = axes[0].imshow(values, cmap='YlOrRd', interpolation='nearest')
axes[0].set_title('Optimal Value Function V*(s)', fontsize=13)
for r in range(GRID_SIZE):
    for c in range(GRID_SIZE):
        axes[0].text(c, r, f'{values[r, c]:.2f}', ha='center', va='center', fontsize=10)
plt.colorbar(im, ax=axes[0])

# Policy arrows
axes[1].set_xlim(-0.5, GRID_SIZE - 0.5)
axes[1].set_ylim(GRID_SIZE - 0.5, -0.5)
axes[1].set_aspect('equal')
axes[1].set_title('Optimal Policy \u03C0*(s)', fontsize=13)
ARROW_DX = [0, 0, -0.3, 0.3]
ARROW_DY = [-0.3, 0.3, 0, 0]
for r in range(GRID_SIZE):
    for c in range(GRID_SIZE):
        if to_state(r, c) == GOAL:
            axes[1].plot(c, r, 'g*', markersize=20)
        else:
            a = int(policy[r, c])
            axes[1].annotate(
                '', 
                xy=(c + ARROW_DX[a], r + ARROW_DY[a]),
                xytext=(c, r),
                arrowprops=dict(arrowstyle='->', color='navy', lw=2)
            )
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Convergence Analysis: VI vs PI

Policy Iteration converges in far fewer iterations (typically in single digits),
while Value Iteration requires many more but each iteration is cheaper.

In [ ]:
pi_result = bellmaneq.solve_policy_iteration(rewards, transitions, gamma=gamma)

print(f'Value Iteration:  {vi_result.iterations} iterations')
print(f'Policy Iteration: {pi_result.iterations} iterations')

# Verify both solvers agree
vi_vals = vi_result.get_values()
pi_vals = pi_result.get_values()
max_diff = np.max(np.abs(vi_vals - pi_vals))
print(f'\nMax |V_vi - V_pi|: {max_diff:.2e}')

In [ ]:
# Convergence curves
histories = {
    'Value Iteration': vi_result.get_convergence_history(),
    'Policy Iteration': pi_result.get_convergence_history(),
}
fig = plot_solver_comparison(histories, title='Convergence: VI vs PI on GridWorld')
plt.show()

## Discount Factor Sensitivity

The discount factor $\gamma$ controls the planning horizon. Higher $\gamma$ means
the agent looks further into the future, but convergence slows down.

In [ ]:
gammas = [0.5, 0.8, 0.9, 0.95, 0.99]

fig, axes = plt.subplots(1, len(gammas), figsize=(4 * len(gammas), 4))

for ax, g in zip(axes, gammas):
    result = bellmaneq.solve_value_iteration(rewards, transitions, gamma=g)
    v = result.get_values().reshape(GRID_SIZE, GRID_SIZE)
    ax.imshow(v, cmap='YlOrRd', interpolation='nearest')
    ax.set_title(f'\u03B3 = {g}\n({result.iterations} iter)', fontsize=11)
    for r in range(GRID_SIZE):
        for c in range(GRID_SIZE):
            ax.text(c, r, f'{v[r, c]:.1f}', ha='center', va='center', fontsize=8)
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle('Effect of Discount Factor on the Value Function', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()